In [1]:
!pip install gymnasium stable-baselines3 moviepy pyvirtualdisplay


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 23.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-man

In [2]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordVideo
import os
from IPython.display import Video

# 创建 CartPole 环境
env = gym.make('CartPole-v1')

# 创建 PPO 模型
model = PPO('MlpPolicy', env, verbose=1)

# 定义一个函数来计算回报
def compute_returns(rewards, gamma):
    returns = []
    G = 0
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    return returns

# 采样多个回合并计算平均回报基线
n_episodes = 10
all_returns = []

for _ in range(n_episodes):
    obs, _ = env.reset()
    done = False
    rewards = []
    while not done:
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)
        rewards.append(reward)
        done = terminated or truncated

    returns = compute_returns(rewards, gamma=0.99)
    total_return = sum(rewards)
    all_returns.append(total_return)

baseline = np.mean(all_returns)
print("Baseline (average return):", baseline)

# 训练模型
model.learn(total_timesteps=100000)

# 保存模型
model.save("ppo_cartpole")

# 加载模型
model = PPO.load("ppo_cartpole")

# 设置视频存储文件夹
video_folder = 'recorded_videos'
os.makedirs(video_folder, exist_ok=True)

# 创建新环境并包装以录制视频
eval_env = gym.make('CartPole-v1', render_mode='rgb_array')
eval_env = RecordVideo(eval_env, video_folder=video_folder, name_prefix='ppo_cartpole')

# 重置环境并开始评估
obs, _ = eval_env.reset()
total_reward = 0
for i in range(1000):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, _ = eval_env.step(action)
    total_reward += reward
    done = terminated or truncated
    if done:
        obs, _ = eval_env.reset()
        print("Total reward for this episode:", total_reward)
        total_reward = 0

eval_env.close()

# 列出生成的视频文件
video_file = [f for f in os.listdir(video_folder) if f.endswith('.mp4')][0]
video_path = os.path.join(video_folder, video_file)
print("Video path:", video_path)

# 显示视频
Video(video_path)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Baseline (average return): 21.8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.8     |
|    ep_rew_mean     | 19.8     |
| time/              |          |
|    fps             | 727      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 28.9        |
|    ep_rew_mean          | 28.9        |
| time/                   |             |
|    fps                  | 536         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010089325 |
|    clip_fraction        | 0.125       |
|    clip_range           | 0

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/recorded_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /content/recorded_videos/ppo_cartpole-episode-0.mp4.
Moviepy - Writing video /content/recorded_videos/ppo_cartpole-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/recorded_videos/ppo_cartpole-episode-0.mp4
Total reward for this episode: 500.0
Moviepy - Building video /content/recorded_videos/ppo_cartpole-episode-1.mp4.
Moviepy - Writing video /content/recorded_videos/ppo_cartpole-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/recorded_videos/ppo_cartpole-episode-1.mp4
Total reward for this episode: 500.0
Video path: recorded_videos/ppo_cartpole-episode-1.mp4
